Imports 

In [1]:
import spacy
from scispacy.abbreviation import AbbreviationDetector
import os
from spacy import displacy
from spacy.pipeline import Tagger
import json 
import csv

Paths to pipeline,models and data

In [2]:
sci_md = '/home/william/Courses/EDAN70/en_core_sci_md-0.2.4/en_core_sci_md/en_core_sci_md-0.2.4'
bionlpg13cg = '/home/william/Courses/EDAN70/en_ner_bionlp13cg_md-0.2.3/en_ner_bionlp13cg_md/en_ner_bionlp13cg_md-0.2.3'
craft = '/home/william/Courses/EDAN70/en_ner_craft_md-0.2.3/en_ner_craft_md/en_ner_craft_md-0.2.3'
bc5cdr = '/home/william/Courses/EDAN70/en_ner_bc5cdr_md-0.2.4/en_ner_bc5cdr_md/en_ner_bc5cdr_md-0.2.4'
path_data = '/home/william/Courses/EDAN70/comm_use_subset_100_new/comm_use_subset_100/'
path_csv = '/home/william/Courses/EDAN70/comm_use_subset_100_new/metadata_comm_use_subset_100.csv'
path_out_bionlpg13cg = '/home/william/Courses/EDAN70/out/bionlpg13cg/'
path_out_craft = '/home/william/Courses/EDAN70/out/craft/'
path_out_bc5cdr = '/home/william/Courses/EDAN70/out/bc5cdr/'

Load Spacy pipeline and models

In [3]:
nlp_sci_md = spacy.load(sci_md)
nlp_bionlpg13cg = spacy.load(bionlpg13cg)
nlp_craft = spacy.load(craft)
nlp_bc5cdr = spacy.load(bc5cdr)

Scan all file names (not necessarily ordered in the same way as in file folder) and put them in an initial data structure

In [4]:
all_files = os.listdir(path_data)
json_files = [pos_json for pos_json in os.listdir(path_data) if pos_json.endswith('.json')]
data = {}
for filename in json_files:
    with open(path_data + filename) as f:
        data[filename] = json.load(f)

Add all texts in a dictionary with their article_id as key and separate title, abstract, and body texts in a better way than before 

In [5]:
structured_data = {}
for key in data:
    texts = []
    abstract = []
    for i in range(len(data[key]['body_text'])): 
        texts.append(data[key]['body_text'][i]['text'])
    for i in range(len(data[key]['abstract'])):
        abstract.append(data[key]['abstract'][i]['text'])
    structured_data[key] = {'title':data[key]['metadata']['title'], 'abstract':abstract, 'texts':texts}

Load CSV data 

In [6]:
keys = structured_data.keys()
csv_data = {}
#print(keys)
with open(path_csv, 'r') as csv_file:
    reader = csv.reader(csv_file)
    line_count = 0
    for row in reader:
        if line_count == 0:
            line_count += 1
        else:
            key = row[1]
            keys_helper = key.split("; ")
            for i in range(len(keys_helper)):
                key_temp = keys_helper[i] + '.json'
                for k in keys: 
                    if(key_temp == k):
                        key = key_temp
                        break
      
            csv_data[key] = {'cord_uid': row[0], 'sourcedb': row[2], 'sourceid': row[5]}
for key in csv_data:
    print(key)
            
        

313d6762ff0c7e18ed7af39482b04fbd2d280bc7.json
406aad9b3f74f619a1aac3f29ad531b28fb7c37e.json
1cdf5f7fb665c7d9efaf8562af501d3b3bdd9273.json
54774da1d5cb436f9bac63726a08b399b05c6992.json
b8423ebcd07e62c48ca7205a06fee753f9199e58.json
a03298211c385b7bd1d1ebdc98b1cc69efe7f54d.json
fd31ade73c720f893a5ddb2370df90a84511ec4a.json
1951545d70e8568083cde219d2583c1aae0c4061.json
88c599b1de8a17cd519f9f72aaaea28392f697f0.json
914c8d326beaccabd37199a9903b27fa7f9b0586.json
f6a6f1fcc991eaf6e733d88adb25a3a7a8e65546.json
849541788c0fe480a2fb9e13b20f3937e759b249.json
5e11e3105842cf7cedf10e26bff170a52c23cf00.json
0b65341c3090421acaac37ec4c93212277be55bb.json
3c67252392670f7eb8bd7ec3c01daf8e2b3a6da9.json
4997e92b014bb26f8184eae26662a09ffcf4f733.json
fae8f01ff5ef7b1737b60dfba0e183bdf482aaba.json
dfbb7bb506552b06794dbcefe56911e2a91a908b.json
5fc3eedb09f3b7624fec7d2b9afedfffbab47a21.json
7d9f6bae2b7ced5834b4e92e7ef805d85fdea405.json
80993091f576dc7fdbec10552b45b4af5eec2b8b.json
2e8bc0044aba50cc5c208dbab92d35493e

Test data structure by printing an article

In [7]:
key = '77c5a33cacec66e3eecf748902a04cd245df6e0f.json'
print(structured_data[key]['title'])
print(len(structured_data[key]['abstract']))
print(len(structured_data[key]['texts']))

TNF-α Acts as an Immunoregulator in the Mouse Brain by Reducing the Incidence of Severe Disease Following Japanese Encephalitis Virus Infection
1
59


Initialize dicts for processed articles and add processed articles for each pipeline/model HEAVY TASK which
is separated into different cells in order to be run on HP Spectre x360 

In [8]:
doc_sci_md = {}
doc_bionlpg13cg = {}
doc_craft = {}
doc_bc5cdr = {}

Show all keys in data 

In [9]:
for key in structured_data:
    print(key)

f2d95ab758e878195eefa64325961667efeb4c14.json
b5af7319ee1fd5e9cc3cf8c5836b19a408ac519b.json
406aad9b3f74f619a1aac3f29ad531b28fb7c37e.json
42a8d5fd943cd29a518ec9683f549f105aac92f8.json
773219a6abb0977144db395a5d94f63ce2eb03c6.json
37142e322bc30493a97e2aff05533919897e39ef.json
c296f6a98b4eb94a69e4caef2df38ccf6e498511.json
68ca53b6414e682661c4decd41eb9ff00574d862.json
372caa549b07492de5fd7064e9cadc62bef0f478.json
4c4dd391e165ce712cb8871ce3082927bbf21ec9.json
dfbb7bb506552b06794dbcefe56911e2a91a908b.json
2a3a68a67776a01043a6a156b69be803c7118a6d.json
fc9a6a00498ee4dfeaf7c201c3341aca6ad21409.json
1cdf5f7fb665c7d9efaf8562af501d3b3bdd9273.json
208f4d527c8ac85c04dd9e431babfaaa54e66428.json
8d14b700a065187eb4d8b01e0e9b6e3e37e5d09b.json
8ec3cfefcf560550c37dbc48107102bf2c893036.json
4fc15d2af497369e0e7aa99dc78a403bdfa4790f.json
73761176155e77d6873806dd9579f59e5419111a.json
fdf03dcd427aa029a6b213288a7e862519decafb.json
8bcd86c97a06dbdff096f6cb164f9dab35f879f9.json
9d2c6b24e096eac147115b77f295e0ddcb

Applying spacy pipeline and all of the models to the data set, separated into different cells with a counter to check progress in loading, 100 represents a completed task. 

In [10]:
n = 0
for key in structured_data:
    title = nlp_sci_md(structured_data[key]['title'])
    abstract = []
    texts = []
                       
    if(len(structured_data[key]['abstract']) != 0):
        for i in range(len(structured_data[key]['abstract'])):
            abstract.append(nlp_sci_md(structured_data[key]['abstract'][i]))
    if(len(structured_data[key]['texts']) != 0):
        for i in range(len(structured_data[key]['texts'])):
            texts.append(nlp_sci_md(structured_data[key]['texts'][i]))
    
    if(len(abstract) == 0 and len(texts) == 0):
        doc_sci_md[key] = {'title':title}
    elif(len(abstract) != 0 and len(texts) == 0):
        doc_sci_md[key] = {'title':title, 'abstract':abstract,}
    elif(len(abstract) == 0 and len(texts) != 0):
        doc_sci_md[key] = {'title':title, 'texts':texts}
    elif(len(abstract) != 0 and len(texts) != 0):
        doc_sci_md[key] = {'title':title,  'abstract':abstract, 'texts':texts}
    
    n = n + 1
    print(n)
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [11]:
n = 0
for key in structured_data:
    title = nlp_bionlpg13cg(structured_data[key]['title'])
    abstract = []
    texts = []
                       
    if(len(structured_data[key]['abstract']) != 0):
        for i in range(len(structured_data[key]['abstract'])):
            abstract.append(nlp_bionlpg13cg(structured_data[key]['abstract'][i]))
    if(len(structured_data[key]['texts']) != 0):
        for i in range(len(structured_data[key]['texts'])):
            texts.append(nlp_bionlpg13cg(structured_data[key]['texts'][i]))
    
    if(len(abstract) == 0 and len(texts) == 0):
        doc_bionlpg13cg[key] = {'title':title}
    elif(len(abstract) != 0 and len(texts) == 0):
        doc_bionlpg13cg[key] = {'title':title, 'abstract':abstract,}
    elif(len(abstract) == 0 and len(texts) != 0):
        doc_bionlpg13cg[key] = {'title':title, 'texts':texts}
    elif(len(abstract) != 0 and len(texts) != 0):
        doc_bionlpg13cg[key] = {'title':title,  'abstract':abstract, 'texts':texts}
    
    n = n + 1
    print(n)
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [12]:
n = 0
for key in structured_data:
    title = nlp_craft(structured_data[key]['title'])
    abstract = []
    texts = []
                       
    if(len(structured_data[key]['abstract']) != 0):
        for i in range(len(structured_data[key]['abstract'])):
            abstract.append(nlp_craft(structured_data[key]['abstract'][i]))
    if(len(structured_data[key]['texts']) != 0):
        for i in range(len(structured_data[key]['texts'])):
            texts.append(nlp_craft(structured_data[key]['texts'][i]))
    
    if(len(abstract) == 0 and len(texts) == 0):
        doc_craft[key] = {'title':title}
    elif(len(abstract) != 0 and len(texts) == 0):
        doc_craft[key] = {'title':title, 'abstract':abstract,}
    elif(len(abstract) == 0 and len(texts) != 0):
        doc_craft[key] = {'title':title, 'texts':texts}
    elif(len(abstract) != 0 and len(texts) != 0):
        doc_craft[key] = {'title':title,  'abstract':abstract, 'texts':texts}
    
    n = n + 1
    print(n)
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [13]:
n = 0
for key in structured_data:
    title = nlp_bc5cdr(structured_data[key]['title'])
    abstract = []
    texts = []
                       
    if(len(structured_data[key]['abstract']) != 0):
        for i in range(len(structured_data[key]['abstract'])):
            abstract.append(nlp_bc5cdr(structured_data[key]['abstract'][i]))
    if(len(structured_data[key]['texts']) != 0):
        for i in range(len(structured_data[key]['texts'])):
            texts.append(nlp_bc5cdr(structured_data[key]['texts'][i]))
    
    if(len(abstract) == 0 and len(texts) == 0):
        doc_bc5cdr[key] = {'title':title}
    elif(len(abstract) != 0 and len(texts) == 0):
        doc_bc5cdr[key] = {'title':title, 'abstract':abstract,}
    elif(len(abstract) == 0 and len(texts) != 0):
        doc_bc5cdr[key] = {'title':title, 'texts':texts}
    elif(len(abstract) != 0 and len(texts) != 0):
        doc_bc5cdr[key] = {'title':title,  'abstract':abstract, 'texts':texts}
    
    n = n + 1
    print(n)
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


Test print

In [14]:
for key in doc_sci_md:
    print(doc_sci_md[key]['title'].ents)

(Zika virus, vaccine, envelope-NS1 polyprotein)
(Inhibition, Putative Dihydropyrimidinase, Pseudomonas aeruginosa, PAO1, Flavonoids, Substrates, Cyclic Amidohydrolases)
(Toll-Like Receptor 3, Respiratory Viral Infections)
(Prevalence, dental caries, associated factors, primary school children, population-based cross-sectional study, Riyadh, Saudi Arabia)
(Stabilized, coronavirus, spikes, resistant, conformational, induced, receptor recognition, proteolysis, OPEN)
(exercises, improve, public health, preparedness, Asia, Middle East, Africa)
(Pathogenic, mechanisms, intravascular coagulation, lungs, BVDV-infected calves, BHV-1)
(Targeting, Nasal Mucosa, Japanese, Encephalitis Virus, Non-Vector-Borne Transmission)
(Citation, Zhao C, Zhao W, NLRP3, Player, Antiviral Responses)
(Detection, Cytomegalovirus Antibodies, Biosensor, Imaging Ellipsometry)
(Assessment of, Knowledge, Attitude, Practice, Prevention, Respiratory Tract Infections, Hajj, Umrah Pilgrims, Malaysia)
(Vpu-interacting Protei

Initialize dictionaries to store tagged entities in

In [15]:
species_bionlpg13cg = {}
chemicals_bionlpg13cg = {}

species_craft = {}
chemicals_craft = {}

disease_bc5cdr = {}
chemicals_bc5cdr = {}      

Tag all entities that are either diseases, chemicals or species for all models.

In [16]:
#BIONLPG13CG
for key in structured_data:
    
    species_bionlpg13cg_title = []
    species_bionlpg13cg_abstract = []
    species_bionlpg13cg_texts = {}

    chemicals_bionlpg13cg_title = []
    chemicals_bionlpg13cg_abstract = []
    chemicals_bionlpg13cg_texts = {}
 

    #Title
    for e in doc_bionlpg13cg[key]['title'].ents:
        if(e.label_ == 'ORGANISM'):
            species_bionlpg13cg_title.append(e)
        if(e.label_ == 'SIMPLE_CHEMICAL'):
            chemicals_bionlpg13cg_title.append(e)

    #Abstract    
    if('abstract' in doc_bionlpg13cg[key].keys()):
        for n in range(len(doc_bionlpg13cg[key]['abstract'])):
            for e in doc_bionlpg13cg[key]['abstract'][n].ents:
                if(e.label_ == 'ORGANISM'):
                    species_bionlpg13cg_abstract.append(e)
                if(e.label_ == 'SIMPLE_CHEMICAL'):
                    chemicals_bionlpg13cg_abstract.append(e)

    #Texts    
    if('texts' in doc_bionlpg13cg[key].keys()):
        for n in range(len(doc_bionlpg13cg[key]['texts'])):
            help_species = []
            help_chemicals = []
            for e in doc_bionlpg13cg[key]['texts'][n].ents:
                if(e.label_ == 'ORGANISM'):
                    help_species.append(e)
                if(e.label_ == 'SIMPLE_CHEMICAL'):
                    help_chemicals.append(e)
            species_bionlpg13cg_texts[n] = help_species
            chemicals_bionlpg13cg_texts[n] = help_chemicals
    
    species_bionlpg13cg[key] = {'title':species_bionlpg13cg_title, 'abstract':species_bionlpg13cg_abstract
                               ,'texts':species_bionlpg13cg_texts}
    chemicals_bionlpg13cg[key] = {'title':chemicals_bionlpg13cg_title, 'abstract':chemicals_bionlpg13cg_abstract
                               ,'texts':chemicals_bionlpg13cg_texts}
    
    for i in range(len(species_bionlpg13cg_texts)):
        section = 'text_' + str(i)
        species_bionlpg13cg[key][section] = species_bionlpg13cg_texts[i]
        
    for i in range(len(chemicals_bionlpg13cg_texts)):
        section = 'text_' + str(i)
        chemicals_bionlpg13cg[key][section] = chemicals_bionlpg13cg_texts[i]
        
            

In [17]:
#CRAFT
for key in structured_data:
    
    species_craft_title = []
    species_craft_abstract = []
    species_craft_texts = {}

    chemicals_craft_title = []
    chemicals_craft_abstract = []
    chemicals_craft_texts = {}
 

    #Title
    for e in doc_craft[key]['title'].ents:
        if(e.label_ == 'TAXON'):
            species_craft_title.append(e)
        if(e.label_ == 'CHEBI'):
            chemicals_craft_title.append(e)

    #Abstract    
    if('abstract' in doc_craft[key].keys()):
        for n in range(len(doc_craft[key]['abstract'])):
            for e in doc_craft[key]['abstract'][n].ents:
                if(e.label_ == 'TAXON'):
                    species_craft_abstract.append(e)
                if(e.label_ == 'CHEBI'):
                    chemicals_craft_abstract.append(e)

    #Texts    
    if('texts' in doc_craft[key].keys()):
        for n in range(len(doc_craft[key]['texts'])):
            help_species = []
            help_chemicals = []
            for e in doc_craft[key]['texts'][n].ents:
                if(e.label_ == 'TAXON'):
                    help_species.append(e)
                if(e.label_ == 'CHEBI'):
                    help_chemicals.append(e)
            species_craft_texts[n] = help_species
            chemicals_craft_texts[n] = help_chemicals
    
    species_craft[key] = {'title':species_craft_title, 'abstract':species_craft_abstract
                               ,'texts':species_craft_texts}
    chemicals_craft[key] = {'title':chemicals_craft_title, 'abstract':chemicals_craft_abstract
                               ,'texts':chemicals_craft_texts}
    
    for i in range(len(species_craft_texts)):
        section = 'text_' + str(i)
        species_craft[key][section] = species_craft_texts[i]
        
    for i in range(len(chemicals_craft_texts)):
        section = 'text_' + str(i)
        chemicals_craft[key][section] = chemicals_craft_texts[i]
                
            

In [18]:
#BC5CDR
for key in structured_data:
    
    disease_bc5cdr_title = []
    disease_bc5cdr_abstract = []
    disease_bc5cdr_texts = {}
 
    chemicals_bc5cdr_title = [] 
    chemicals_bc5cdr_abstract = [] 
    chemicals_bc5cdr_texts = {} 

    #Title
    for e in doc_bc5cdr[key]['title'].ents:
        if(e.label_ == 'DISEASE'):
             disease_bc5cdr_title.append(e)
        if(e.label_ == 'CHEMICAL'):
            chemicals_bc5cdr_title.append(e)

    #Abstract    
    if('abstract' in doc_bc5cdr[key].keys()):
        for n in range(len(doc_bc5cdr[key]['abstract'])):
            for e in doc_bc5cdr[key]['abstract'][n].ents:
                if(e.label_ == 'DISEASE'):
                     disease_bc5cdr_abstract.append(e)
                if(e.label_ == 'CHEMICAL'):
                    chemicals_bc5cdr_abstract.append(e)

    #Texts    
    if('texts' in doc_bc5cdr[key].keys()):
        for n in range(len(doc_bc5cdr[key]['texts'])):
            help_disease = []
            help_chemicals = []
            for e in doc_bc5cdr[key]['texts'][n].ents:
                if(e.label_ == 'DISEASE'):
                    help_disease.append(e)
                if(e.label_ == 'CHEMICAL'):
                    help_chemicals.append(e)
            disease_bc5cdr_texts[n] = help_disease
            chemicals_bc5cdr_texts[n] = help_chemicals
            
    disease_bc5cdr[key] = {'title': disease_bc5cdr_title, 'abstract': disease_bc5cdr_abstract
                               ,'texts': disease_bc5cdr_texts}
    chemicals_bc5cdr[key] = {'title':chemicals_bc5cdr_title, 'abstract':chemicals_bc5cdr_abstract
                               ,'texts':chemicals_bc5cdr_texts}
    
    for i in range(len(disease_bc5cdr_texts)):
        section = 'text_' + str(i)
        disease_bc5cdr[key][section] = disease_bc5cdr_texts[i]
        
    for i in range(len(chemicals_bc5cdr_texts)):
        section = 'text_' + str(i)
        chemicals_bc5cdr[key][section] = chemicals_bc5cdr_texts[i]

                
            

In [19]:
key = '80993091f576dc7fdbec10552b45b4af5eec2b8b.json'
print(species_bionlpg13cg[key]['title'])
print(len(species_bionlpg13cg[key]['abstract'][0]))
e = (species_bionlpg13cg[key]['texts'][3][0])
for x in e:
    print(x.text, x.ent_type_, x.ent_iob_)

[]
2
GMT-5 ORGANISM B


In [20]:
for p in chemicals_bc5cdr[key]['abstract']:
    print(p.text, p.start_char, p.end_char, p.label_)

print("NEW")

for x in chemicals_bc5cdr[key]['abstract']:
    for e in x:
        print(e.text,e.ent_type_, e.ent_iob_)

NEW


In [21]:
abstracts = {}
for key in structured_data:
    abstracts[key] = ""
    for i in range(len(structured_data[key]['abstract'])):
        abstracts[key] = abstracts[key] + structured_data[key]['abstract'][i]

### FILE WRITER ### 

In [22]:
def addDenotation(string, e, model, object, id):
    string = string + "\n    {"
    string = string + "\n      \"id\": \"" + model + "_T" + str(id) + "\","
    string = string + "\n      \"span\": {"
    string = string + "\n        \"begin\": " + str(e.start_char) + "," 
    string = string + "\n        \"end\": " + str(e.end_char)
    string = string + "\n      },"
    string = string + "\n      \"obj\": \"" + object + "\""
    string = string + "\n    }"
    return string

In [23]:
def stringBuilder(string, e, model, object, id): 
    string = addDenotation(string, e, model, object, id)
    string = string + ","            
    return string

In [24]:
def addDenotations(string, key, section, model):
    id = 1
    bool = False
    string = string + "\n  \"denotations\": ["
    if(model == 'bionlpg13cg'):
        for e in species_bionlpg13cg[key][section]:
            string = stringBuilder(string, e, model, 'species', id)
            id = id + 1    
            bool = True
        for e in chemicals_bionlpg13cg[key][section]:
            string = stringBuilder(string, e, model, 'chemical', id)
            id = id + 1
            bool = True
            
    if(model == 'craft'):  
        for e in species_craft[key][section]:
            string = stringBuilder(string, e, model, 'species', id)
            id = id + 1   
            bool = True
        for e in chemicals_craft[key][section]:
            string = stringBuilder(string, e, model, 'chemical', id)
            id = id + 1
            bool = True
            
    if(model == 'bc5cdr'):
        for e in disease_bc5cdr[key][section]:
            string = stringBuilder(string, e, model, 'disease', id)
            id = id + 1         
            bool = True
        for e in chemicals_bc5cdr[key][section]:
            string = stringBuilder(string, e, model, 'chemical', id)
            id = id + 1
            bool = True
   
    if(bool):
        string = string[:-1]
        string = string + "\n  ] \n}"
    else:
        string = string + "]\n}"
    return string

In [25]:
def writer(key, id):
    string = "{ \n  \"cord_uid\": \"" + csv_data[key]['cord_uid'] + "\","
    string = string + "\n  \"sourcedb\": \"" + csv_data[key]['sourcedb'] + "\","
    string = string + "\n  \"sourceid\": \"" + csv_data[key]['sourceid'] + "\","
    string = string + "\n  \"divid\": \"" + str(id) + "\","
    return string 

In [26]:
def writerTitle(key, section, model):
    string = "\n  \"text\": \"" + structured_data[key][section] + "\","
    string = string + "\n  \"project\": \"cdlai_CORD-19\","
    string = addDenotations(string, key, section, model)
    return string

In [27]:
def writerAbstract(key, section, model):
    string = "\n  \"text\": \"" + abstracts[key] + "\","
    string = string + "\n  \"project\": \"cdlai_CORD-19\","
    string = addDenotations(string, key, section, model)
    return string

In [28]:
def writerBody(key, section, model, n):
    string = "\n  \"text\": \"" + structured_data[key]['texts'][n] + "\","
    string = string + "\n  \"project\": \"cdlai_CORD-19\","
    string = addDenotations(string, key, section, model)
    return string

In [31]:
def out(path, model):
    for key in structured_data:
        name_title = csv_data[key]['cord_uid'] + '-0-title.json'
        with open(path + name_title, 'w') as file:
            string = writer(key, 0) + writerTitle(key, 'title', model)
            file.write(string)

        if(len(structured_data[key]['abstract']) != 0):
            name_abstract = csv_data[key]['cord_uid'] + '-1-abstract.json'
            with open(path + name_abstract, 'w') as file:
                string = writer(key, 1) + writerAbstract(key, 'abstract', model)
                file.write(string)

        for i in range(len(structured_data[key]['texts'])):
            n = i + 2
            name_text = csv_data[key]['cord_uid'] + '-' + str(n) + '-body_text.json'
            section = 'text_' + str(i)
            with open(path + name_text, 'w') as file:
                string = writer(key, n) + writerBody(key, section, model, i)
                file.write(string)

In [33]:
out(path_out_bionlpg13cg, 'bionlpg13cg')
out(path_out_craft, 'craft')
out(path_out_bc5cdr, 'bc5cdr')